<a href="https://colab.research.google.com/github/rayaneghilene/BERT_Hate_Classification/blob/main/Arabic_BERT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of the performance BERT classifiers on Arabic text data

Note: the model requires mode storage for processing arabic data, than it does for english and french.

Modify the following variables based on your data
* **DATAPATH**: is the path to your dataset in your environment

In [1]:
DATAPATH = '/content/ar_reviews_100k.tsv'

## Data  

In [2]:
# Install and import the required libraries
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off


In [3]:
!pip install nltk --progress-bar off
!pip install keras --progress-bar off
!pip install tensorflow --progress-bar off
!pip install tensorflow_hub --progress-bar off
!pip install transformers --progress-bar off

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

In [5]:
#creation of mapped labels
df = pd.read_csv(DATAPATH, sep='\t')
max_features = 10000
sequence_length = 250
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

vectorize_layer.adapt(df['text'])
label_mapping = {subtype: label for label, subtype in enumerate(df['label'].unique())}
num_classes = len(label_mapping)

batch_size = 16
def vectorize_text(text, label):
    return vectorize_layer(text), label

df['label'] = df['label'].map(label_mapping)

columns = ['text', 'label']
new_df = df[columns].copy()

new_df1, nan = train_test_split(new_df, test_size=0.9999, stratify=df['label'], random_state=42)


csv_file_path = '/content/arabic_dataset.csv'
new_df1.to_csv(csv_file_path, index=False)

## I - RoBERTa

In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

df = pd.read_csv(csv_file_path)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True, return_tensors="tf")
val_encodings = tokenizer(list(val_df['text']), truncation=True, padding=True, return_tensors="tf")

train_labels = tf.convert_to_tensor(list(train_df['label']))
val_labels = tf.convert_to_tensor(list(val_df['label']))

model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=8)

optimizer = Adam(learning_rate=5e-5)
loss_fn = SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))

train_dataset = train_dataset.batch(16).shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=10, validation_data=val_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification

Epoch 1/10
1/1 [==============================] - 55s 55s/step - loss: 2.1479 - accuracy: 0.1429 - val_loss: 2.0593 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 1s 935ms/step - loss: 2.0207 - accuracy: 0.1429 - val_loss: 2.0102 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 1s 869ms/step - loss: 1.8546 - accuracy: 0.4286 - val_loss: 1.9781 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 1s 881ms/step - loss: 1.6985 - accuracy: 0.4286 - val_loss: 1.9451 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 1s 884ms/step - loss: 1.5389 - accuracy: 0.4286 - val_loss: 1.8863 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 1s 842ms/step - loss: 1.3206 - accuracy: 0.4286 - val_loss: 1.6999 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 1s 866ms/step - loss: 1.1890 - accuracy: 0.4286 - val_loss: 1.6757 - val_accuracy: 0.00

In [7]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')



custom_prompt = "جمبل جدا "


inputs = tokenizer(custom_prompt, return_tensors="tf")
outputs = model(inputs["input_ids"])
predictions = tf.nn.softmax(outputs.logits, axis=-1).numpy()
predicted_class_index = tf.argmax(predictions, axis=-1).numpy()[0]

predicted_class_name = next(subtype for subtype, label in label_mapping.items() if label == predicted_class_index)
print("Predicted Class Name:", predicted_class_name)

Predicted Class Name: Mixed


## II - ELECTRA

In [8]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import ElectraTokenizerFast, TFElectraForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

df = pd.read_csv(csv_file_path)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
tokenizer = ElectraTokenizerFast.from_pretrained('google/electra-base-discriminator')
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True, return_tensors="tf")
val_encodings = tokenizer(list(val_df['text']), truncation=True, padding=True, return_tensors="tf")

train_labels = tf.convert_to_tensor(list(train_df['label']))
val_labels = tf.convert_to_tensor(list(val_df['label']))

model = TFElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=8)

optimizer = Adam(learning_rate=5e-5)
loss_fn = SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))

train_dataset = train_dataset.batch(16).shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(64)

model.fit(train_dataset, epochs=3, validation_data=val_dataset)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1/1 [==============================] - 50s 50s/step - loss: 2.0761 - accuracy: 0.1429 - val_loss: 2.0420 - val_accuracy: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 1s 920ms/step - loss: 2.0731 - accuracy: 0.2857 - val_loss: 1.9845 - val_accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 1s 873ms/step - loss: 1.9677 - accuracy: 0.7143 - val_loss: 1.9279 - val_accuracy: 0.5000


## III - DistilBERT

In [9]:
import tensorflow as tf
import pandas as pd
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split

# Data preprocessing
df = pd.read_csv(csv_file_path)
df["label"] = df["label"]
data_texts = df["text"].to_list()
data_labels = df["label"].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(data_texts, data_labels, test_size=0.2)

# Tokenisation
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

# DistilBERT Model
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)  # Replace 8 with your number of classes

# Optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


train_dataset = train_dataset.batch(16)
test_dataset = test_dataset.batch(16)

# Training
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Save the model
model.save_pretrained("./distilbert-fine-tuned")

# Load the fine-tuned model
model = TFDistilBertForSequenceClassification.from_pretrained("./distilbert-fine-tuned")


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
1/1 [==============================] - 24s 24s/step - loss: 2.1430 - accuracy: 0.0000e+00 - val_loss: 2.0418 - val_accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 0s 416ms/step - loss: 2.0314 - accuracy: 0.1429 - val_loss: 1.9580 - val_accuracy: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 318ms/step - loss: 1.9020 - accuracy: 0.4286 - val_loss: 1.8702 - val_accuracy: 0.0000e+00


Some layers from the model checkpoint at ./distilbert-fine-tuned were not used when initializing TFDistilBertForSequenceClassification: ['dropout_95']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert-fine-tuned and are newly initialized: ['dropout_115']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###Test on a Custom Prompt